$$\text{Parallélisation d'une décomposition de Cholesky}$$

$$\text{Florian Schirrer _ Caroline Boudier }$$

$$\textit{Éleménts Logiciels pour le traitement des Données Massives 2021}$$

# Pré-requis

In [ ]:
# packages
from sklearn.datasets import make_spd_matrix
from numpy.testing import assert_almost_equal
import tensorflow as tf
import numpy as np
import cython
%load_ext cython
import matplotlib.pyplot as plt
import pandas as pd

# I) Fonctions Python

On va coder en Python deux versions classiques de la décomposition de Cholesky : 
- Cholesky-Banachiewicz
- Cholesky-Crout. 


## A) Cholesky-Banachiewicz

In [ ]:
def python_Bana(A):
  # taille de la matrice
  n=len(A)
  # on crée une matrice vide
  L=np.zeros((n,n))
  # on parcourt ses lignes
  for i in range(n):
    # puis ses colonnes
    for j in range(i+1):
      s=0
      # calcul de la somme
      for k in range(j):
        s+=L[i][k]*L[j][k]
      if (i==j):
        L[i][j]=(A[i][i]-s)**0.5
      else:
        L[i][j]=(1.0/L[j][j])*(A[i][j]-s)
  return(L)

In [ ]:
N=make_spd_matrix(n_dim=100)
# on vérifie le résultat - fonction baseline = tensorflow
assert_almost_equal(tf.linalg.cholesky(N),python_Bana(N),decimal=5)

# on compare les temps pour se faire une première idée
print('temps Python Bana')
%timeit -o python_Bana(N)
print('temps baseline tensorflow')
%timeit -o tf.linalg.cholesky(N)

## B) Cholesky-Crout

In [ ]:
def python_Crout(A):
    n=len(A)
    # on crée une matrice vide
    L = np.zeros((n,n))
    # on parcourt ses colonnes
    for j in range(n):
        s=0.0
        for k in range(j):
            s+=L[j][k]*L[j][k]
        L[j][j]=np.sqrt((A[j][j]-s))
        # puis ses lignes
        for i in range(j+1,n):
            s = 0
            for k in range(j):
                s+=L[i][k]*L[j][k]
            
            L[i][j]=(1.0/L[j][j])*(A[i][j]-s)
    return(L)

In [ ]:
N=make_spd_matrix(n_dim=100)
# on vérifie le résultat
assert_almost_equal(tf.linalg.cholesky(N),python_Crout(N),decimal=5)

# on compare les temps pour se faire une première idée
print('temps Python Crout')
%timeit -o python_Crout(N)
print('temps baseline tensorflow')
%timeit -o tf.linalg.cholesky(N)

# II) Parallélisation de l'algorithme des colonnes (Cholesky-Crout)

Comme décrit dans le rapport nous décidons de paralléliser la version Cholesky-Crout de l'algorithme plutôt que la version Cholesky Banachiewicz.

### A) Premières optimisations (indépendantes d'une parallélisation). 

On commence par compiler le code avec Cython en apportant plusieurs améliorations : 
- on définit les types de variables (afin d'éviter de créer des objets Python)
- on crée des memory views qui pointeront vers nos arrays numpy.
- on enlève le "bounds checking" (vérifie que les indices sont cohérents) et le wraparound, on intègre aussi une division de type C. 



In [ ]:
%%cython -a
import numpy as np
cimport cython 

@cython.boundscheck(False) # retire le boundschecking
@cython.wraparound(False) # retire le wraparound
@cython.cdivision(True) # division de type C

def cython_cholesky_crout(double[:,:] A):
    # on définit les types de valeurs pour i,j,k,n, s
    cdef Py_ssize_t n=len(A)
    cdef Py_ssize_t i,j,k
    cdef float s
    
    # on initialise à 0 une matrice L
    L = np.zeros((n,n))
    
    # on crée des memory views
    cdef double[:,:] L_view=L
    cdef double[:,:] A_view=A

    # on boucle sur les colonnes
    for j in range(n):
        s=0.0
        for k in range(j):
            s+=L_view[j][k]*L_view[j][k]
        
        L_view[j][j]=(A_view[j][j]-s)**0.5
        # puis sur les lignes
        for i in range(j+1,n):
            s = 0
            for k in range(j):
                s+=L_view[i][k]*L_view[j][k]
            
            L_view[i][j]=(1.0/L_view[j][j])*(A_view[i][j]-s)

    return(L)

In [ ]:
N=make_spd_matrix(n_dim=100)
# on vérifie le résultat
assert_almost_equal(cython_cholesky_crout(N),tf.linalg.cholesky(N),decimal=5)

# on compare les temps
print('fonction Python de base: ')
%timeit -o python_Crout(N)
print('fonction Python compilée en Cython avec optimisations')
%timeit -o cython_cholesky_crout(N)

### B) Parallélisation

On va maintenant paralléliser la fonction. À noter que notre fonction prend en entrée : la matrice, le nombre de threads (fixé par défaut à une valeur de 8) et un cut-off (fixé par défaut à 0). Le cutoff est ici défini comme la proportion non parallélisée de l'algorithme, autrement dit si on fixe un cutoff à 20%, la parallélisation se fera sur les 80% des premières colonnes et le reste en séquentiel. 

In [ ]:
%%cython

# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp

import numpy as np
from cython.parallel cimport prange
cimport cython 

@cython.boundscheck(False) 
@cython.wraparound(False) 
@cython.cdivision(True) 

def cython_cholesky_crout_par(double[:,:] A,int n_threads=8,float cutoff=0):
    cdef Py_ssize_t n=len(A)
    cdef Py_ssize_t i,j,k
    cdef float s
    L = np.zeros((n,n))
    cdef double[:,:] L_view=L
    cdef double[:,:] A_view=A
    for j in range(n):
        s=0.0
        for k in range(j):
            s+=L_view[j][k]*L_view[j][k]
        
        L_view[j][j]=(A_view[j][j]-s)**0.5
        
        # la parallélisation intervient ici
        if j<n-int(n*cutoff):
            for i in prange(j+1,n,nogil=True,schedule='static',num_threads=n_threads):
                s = 0
                for k in range(j):
                    s=s+L_view[i][k]*L_view[j][k]
            
                L_view[i][j]=(1.0/L_view[j][j])*(A_view[i][j]-s)
        else:
            for i in range(j+1,n):
                s = 0
                for k in range(j):
                    s=s+L_view[i][k]*L_view[j][k]
            
                L_view[i][j]=(1.0/L_view[j][j])*(A_view[i][j]-s)
            
    return(L)

In [ ]:
N=make_spd_matrix(1000)
# on vérifie le résultat
assert_almost_equal(cython_cholesky_crout_par(N),tf.linalg.cholesky(N),decimal=3)

# on affiche quelques temps pour se faire une première idée
print('Cholesky Crout non parallélisé')
%timeit cython_cholesky_crout(N)
print('Cholesky Crout parallélisé - 2 threads - pas de cutoff')
%timeit cython_cholesky_crout_par(N,n_threads=2)
print('Cholesky Crout parallélisé - 8 threads - pas de cutoff')
%timeit cython_cholesky_crout_par(N,n_threads=8)
print('Cholesky Crout parallélisé - 8 threads -cutoff de 50%')
%timeit cython_cholesky_crout_par(N,n_threads=8,cutoff=0.5)

# III) Parallélisation d'un algorithme optimisé (vecteurs plutôt que matrices). 

## A) Version non parallélisée

On implémente maintenant la version contigüe en représentant nos matrices par des vecteurs. Afin d'accélérer nos calculs, nous ne passons pas par un mermoyview, mais nous introduisons deux pointeurs: un sur la matrice A et un sur la matrice L

In [ ]:
%%cython -a
import numpy as np
cimport cython 
cimport numpy as np
from cpython cimport array

@cython.boundscheck(False) # retire le boundschecking
@cython.wraparound(False) # retire le wraparound
@cython.cdivision(True) # division de type C

def cython_cholesky_crout_vect(np.ndarray A):
    # on définit les types de valeurs pour i,j,k,n, s
    cdef Py_ssize_t n=len(A)
    cdef Py_ssize_t i,j,k
    cdef double s
    
    #On représente la matrice 2D sous la forme d'un vecteur 1D
    cdef np.ndarray A_reshape = A.reshape(n*n)
    
    #Pointeur sur la forme contigüe de L
    cdef np.ndarray[double, ndim=1, mode = 'c'] L = np.zeros(n*n)
    cdef double* L_view = <double*> L.data

    #Pointeur sur la forme contigüe de A
    cdef double* A_view = <double*> A_reshape.data
    for j in range(n):
        s=0.0
        for k in range(j):
            s+=L_view[j*n+k]*L_view[j*n+k]
        
        L_view[j*n+j]=(A_view[j*n+j]-s)**0.5
        
        for i in range(j+1,n):
            s = 0
            for k in range(j):
                s+=L_view[i*n+k]*L_view[j*n+k]
            
            L_view[i*n+j]=(1.0/L_view[j*n+j])*(A_view[i*n+j]-s)

    return(L.reshape((n,n)))

In [ ]:
N=make_spd_matrix(n_dim=1000)
# on vérifie le résultat
assert_almost_equal(cython_cholesky_crout_vect(N),tf.linalg.cholesky(N),decimal=3)
print('Cholesky Crout simple :')
%timeit -o cython_cholesky_crout(N)
print('Cholesky vectorisé simple')
%timeit -o cython_cholesky_crout_vect(N)

## B) Version parallélisée

On crée une seconde version paralllélisée. De même que pour l'algorithme non vectorisé on prend en compte le nombre de threads et le cutoff. 

In [ ]:
%%cython -a
# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp
import numpy as np
from cython.parallel cimport prange
cimport cython 
cimport numpy as np
from cpython cimport array

@cython.boundscheck(False) # retire le boundschecking
@cython.wraparound(False) # retire le wraparound
@cython.cdivision(True) # division de type C

def cython_cholesky_crout_vect_par(np.ndarray A,int n_threads=8,float cutoff=0):
    # on définit les types de valeurs pour i,j,k,n, s
    cdef Py_ssize_t n=len(A)
    cdef Py_ssize_t i,j,k
    cdef double s
    cdef np.ndarray A_reshape = A.reshape(n*n)
    cdef np.ndarray[double, ndim=1, mode = 'c'] L = np.zeros(n*n)
    
    # création de pointeurs
    cdef double* L_view = <double*> L.data
    cdef double* A_view = <double*> A_reshape.data
    for j in range(n):
        s=0.0
        for k in range(j):
            s+=L_view[j*n+k]*L_view[j*n+k]
        
        L_view[j*n+j]=(A_view[j*n+j]-s)**0.5
        
        #parallélisation avec cutoff
        if j<n-int(n*cutoff):
          for i in prange(j+1,n,nogil=True,schedule='static',num_threads=n_threads):
            s = 0
            for k in range(j):
              s=s+L_view[i*n+k]*L_view[j*n+k]
            L_view[i*n+j]=(1.0/L_view[j*n+j])*(A_view[i*n+j]-s)
        else:
          for i in range(j+1,n):
            s = 0
            for k in range(j):
              s=s+L_view[i*n+k]*L_view[j*n+k]
            L_view[i*n+j]=(1.0/L_view[j*n+j])*(A_view[i*n+j]-s)

    return(L.reshape((n,n)))

In [ ]:
N=make_spd_matrix(1000)
# on vérifie le résultat
assert_almost_equal(cython_cholesky_crout_vect_par(N),tf.linalg.cholesky(N),decimal=3)

# on affiche quelques temps pour se faire une première idée
print('Cholesky Crout vectorisé non parallélisé')
%timeit cython_cholesky_crout_vect(N)
print('Cholesky Crout vectorisé parallélisé - 2 threads - pas de cutoff')
%timeit cython_cholesky_crout_vect_par(N,n_threads=2)
print('Cholesky Crout vectorisé parallélisé - 8 threads - pas de cutoff')
%timeit cython_cholesky_crout_vect_par(N,n_threads=8)
print('Cholesky Crout vectorisé parallélisé - 8 threads -cutoff de 50%')
%timeit cython_cholesky_crout_vect_par(N,n_threads=8,cutoff=0.5)

# IV) Comparaison des temps et visualisation. 

L'idée va être : 

- de comparer la version parallélisée et non parallélisée en calculant les temps d'exécution pour différentes tailles de matrices, les speed-ups, l'efficacité et la variation en fonction de certains paramètres (nombre de threads, cut-offs). 



## A) Temps d'exécution et speed-up en fonction de la taille de la matrice

Les temps d'exécution sont pour l'instant calculés pour une utilisation maximale des capacités (8 threads) et sans cut-off. 

In [ ]:
vect=[100,500,750,1000,2500,5000]
threads_def=8
cutoff_def=0.0

# dictionnaire d'arrivée
result_dict={'matrix_size':vect,
             'CC_simple':[],
             'CC_par':[],
             'speed_up_CC':[],
             'vect_simple':[],
             'vect_par':[],
             'vect_speedup':[],
             'baseline_tensorflow':[]
             }

for n in vect:
  # on crée une matrice de taille N
  A=make_spd_matrix(n)
  
  # temps pour l'algo colonnes
    #col 
  t_col=%timeit -o -r 3 cython_cholesky_crout(A)
  result_dict['CC_simple'].append(t_col.best)
   #col par
  t_col_par= %timeit -o -r 3 cython_cholesky_crout_par(A,n_threads=threads_def,cutoff=cutoff_def)
  result_dict['CC_par'].append(t_col_par.best)
    #col speed_up
  result_dict['speed_up_CC'].append(t_col.best/t_col_par.best)

  # temps pour vecteur 
    # vecteur normal 
  t_vect= %timeit -o -r 3 cython_cholesky_crout_vect(A)
  result_dict['vect_simple'].append(t_vect.best)
    # vecteur parallélisé 
  t_vect_par= %timeit -o -r 3 cython_cholesky_crout_vect_par(A,n_threads=threads_def,cutoff=cutoff_def)
  result_dict['vect_par'].append(t_vect_par.best)
    # speed-up vecteur 
  result_dict['vect_speedup'].append(t_vect.best/t_vect_par.best)

  # temps pour tensorflow
  tf_time= %timeit -o -r 3 tf.linalg.cholesky(A)
  result_dict['baseline_tensorflow'].append(tf_time.best)

In [ ]:
# on affiche les résultats
results_df=pd.DataFrame.from_dict(result_dict)
results_df=results_df.set_index('matrix_size')
results_df.round(3)

In [ ]:
#graphique pour les temps d'exécution et pour les speed-ups
results_df.plot(y=['CC_simple','CC_par','vect_simple','vect_par','baseline_tensorflow'],title='Temps d\'exécution pour nos différents algorithmes en fonction de la taille de la matrice')
results_df.plot(y=['speed_up_CC','vect_speedup'],title='Speed-Up pour nos deux algorithmes',kind='bar')

## B) Impact du nombre de threads et du cut-off

Impact du nombre de threads : on va calculer l'évolution du speed-up et de l'efficacité pour une matrice de taille définie. 

In [ ]:
# Plot pour le nombre de threads
mat=make_spd_matrix(3000)

n_threads=[1,2,4,6,8]

# dictionnaire d'arrivée
result_threads_dict={}

for i in n_threads:
  result_threads_dict[i]=[]
  #time_col
  t_col=%timeit -o cython_cholesky_crout_par(mat,n_threads=i)
    #time
  result_threads_dict[i].append(t_col.best)
    #speed-up
  result_threads_dict[i].append(result_threads_dict[1][0]/t_col.best)
    #efficieny
  result_threads_dict[i].append(result_threads_dict[i][1]/i)

  #time_vect
  t_vect=%timeit -o cython_cholesky_crout_vect_par(mat,n_threads=i)
    #time
  result_threads_dict[i].append(t_vect.best)
    #speed-up
  result_threads_dict[i].append(result_threads_dict[1][3]/t_vect.best)
    # efficiency
  result_threads_dict[i].append(result_threads_dict[i][4]/i)

In [ ]:
# on affiche nos résultats
results_th_df=pd.DataFrame.from_dict(result_threads_dict,orient='index',columns=['Temps Cholesky-Crout','Speed-up CC','Efficacité CC','Temps Vecteur','Speed-Up Vecteur','Efficacité Vecteur'])
results_th_df.index.name='Nombre de threads'
results_th_df=results_th_df.round(2)
results_th_df

In [ ]:
# on trace un graphique pour représenter efficacité et speed-up en fonction du nombre de threads
results_th_df.plot(y=['Speed-up CC','Speed-Up Vecteur'],title='Speed-Up pour nos deux algorithmes en fonction du nombre de threads',xlabel='Nombre de threads',kind='bar',rot=0)
#plt.plot(results_th_df['Crout_speed_up'],label=n_threads)
results_th_df.plot(y=['Efficacité CC','Efficacité Vecteur'],title='Efficacité pour nos deux algorithmes en fonction du nombre de threads',xlabel='Nombre de threads',rot=0)

Impact du cut-off : on va calculer l'évolution du temps d'exécution en fonction du cut-off pour deux matrices de taille différente. 

In [ ]:
small_mat=make_spd_matrix(50)
big_mat=make_spd_matrix(300)
cutoff_range = [i/10 for i in range(11)]

co_dict={'cut_off':cutoff_range,
         'small_mat_CC':[],
         'big_mat_CC':[],
         'small_mat_vect':[],
         'big_mat_vect':[]
         }

for co in cutoff_range:
  print(co)
  # columns
  t_small=%timeit -o cython_cholesky_crout_par(small_mat,cutoff=co,n_threads=2)
  co_dict['small_mat_CC'].append(t_small.best)
  t_big=%timeit -o cython_cholesky_crout_par(big_mat,cutoff=co,n_threads=2)
  co_dict['big_mat_CC'].append(t_big.best)
  # vect
  t_small_vect=%timeit -o cython_cholesky_crout_vect_par(small_mat,cutoff=co,n_threads=2)
  co_dict['small_mat_vect'].append(t_small_vect.best)
  t_big_vect=%timeit -o cython_cholesky_crout_vect_par(big_mat,cutoff=co,n_threads=2)
  co_dict['big_mat_vect'].append(t_big_vect.best)

In [ ]:
results_th_df=pd.DataFrame.from_dict(co_dict)
results_th_df

In [ ]:
results_th_df.plot(x='cut_off',y=['small_mat_CC','small_mat_vect'],title='Évolution du temps d\'exécution pour une matrice de taille 50*50')
results_th_df.plot(x='cut_off',y=['big_mat_CC','big_mat_vect'],title='Évolution du temps d\'exécution pour une matrice de taille 300*300')